In [1]:
pip install tensorflow keras 


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import keras.backend as K
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Input, LeakyReLU
from keras.models import Model
from keras.optimizers import Adam
from tqdm import tqdm

from solver import action_map_small, gen_sequence, get_all_possible_actions_cube_small, chunker, \
    flatten_1d_b
    

In [6]:
def acc(y_true, y_pred):
    return K.cast(K.equal(K.max(y_true, axis=-1),
                          K.cast(K.argmax(y_pred, axis=-1), K.floatx())),
                  K.floatx())

In [ ]:
def get_model(lr=0.0001):
    input1 = Input((324,))

    d1 = Dense(1024)
    d2 = Dense(1024)
    d3 = Dense(1024)

    d4 = Dense(50)

    x1 = d1(input1)
    x1 = LeakyReLU()(x1)
    x1 = d2(x1)
    x1 = LeakyReLU()(x1)
    x1 = d3(x1)
    x1 = LeakyReLU()(x1)
    x1 = d4(x1)
    x1 = LeakyReLU()(x1)

    out_value = Dense(1, activation="linear", name="value")(x1)
    out_policy = Dense(len(action_map_small), activation="softmax", name="policy")(x1)

    model = Model(input1, [out_value, out_policy])

    model.compile(loss={"value": "mae", "policy": "sparse_categorical_crossentropy"}, optimizer=Adam(lr),
                  metrics={"policy": acc})
    model.summary()

    return model


if __name__ == "__main__":

    N_SAMPLES = 100
    N_EPOCH = 10000

    file_path = "auto.h5"

    checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    early = EarlyStopping(monitor="val_loss", mode="min", patience=1000)

    reduce_on_plateau = ReduceLROnPlateau(monitor="val_loss", mode="min", factor=0.1, patience=50, min_lr=1e-8)

    callbacks_list = [checkpoint, early, reduce_on_plateau]

    model = get_model(lr=0.0001)
    #model.load_weights(file_path)

    for i in range(N_EPOCH):
        cubes = []
        distance_to_solved = []
        for j in tqdm(range(N_SAMPLES)):
            _cubes, _distance_to_solved = gen_sequence(25)
            cubes.extend(_cubes)
            distance_to_solved.extend(_distance_to_solved)

        cube_next_reward = []
        flat_next_states = []
        cube_flat = []

        for c in tqdm(cubes):
            flat_cubes, rewards = get_all_possible_actions_cube_small(c)
            cube_next_reward.append(rewards)
            flat_next_states.extend(flat_cubes)
            cube_flat.append(flatten_1d_b(c))

        for _ in range(20):

            cube_target_value = []
            cube_target_policy = []

            next_state_value, _ = model.predict(np.array(flat_next_states), batch_size=1024)
            next_state_value = next_state_value.ravel().tolist()
            next_state_value = list(chunker(next_state_value, size=len(action_map_small)))

            for c, rewards, values in tqdm(zip(cubes, cube_next_reward, next_state_value)):
                r_plus_v = 0.4*np.array(rewards) + np.array(values)
                target_v = np.max(r_plus_v)
                target_p = np.argmax(r_plus_v)
                cube_target_value.append(target_v)
                cube_target_policy.append(target_p)

            cube_target_value = (cube_target_value-np.mean(cube_target_value))/(np.std(cube_target_value)+0.01)

            print(cube_target_policy[-30:])
            print(cube_target_value[-30:])

            sample_weights = 1. / np.array(distance_to_solved)
            sample_weights = sample_weights * sample_weights.size / np.sum(sample_weights)

            model.fit(np.array(cube_flat), [np.array(cube_target_value), np.array(cube_target_policy)[..., np.newaxis]],
                      epochs=1, batch_size=128, sample_weight=[sample_weights, sample_weights])
            # sample_weight=[sample_weights, sample_weights],

        model.save_weights(file_path)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 324)]        0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 1024)         332800      ['input_2[0][0]']                
                                                                                                  
 leaky_re_lu_4 (LeakyReLU)      (None, 1024)         0           ['dense_4[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 1024)         1049600     ['leaky_re_lu_4[0][0]']          
                                                                                            

100%|██████████| 2500/2500 [01:07<00:00, 36.96it/s]


30/30 [==============================] - 1s 16ms/step


2500it [00:00, 96163.46it/s]

[6, 4, 8, 0, 6, 10, 4, 4, 10, 8, 6, 11, 7, 5, 11, 10, 4, 7, 4, 11, 2, 4, 4, 1, 4, 10, 4, 9, 0, 9]
[-0.54438774  0.00784822  3.65527477 -0.1122011   3.65527477 -0.57913239
  0.37889508 -0.07943006 -1.03612266 -0.12261677 -0.11082152 -0.12261677
 -0.11736421 -0.12261677 -0.10477907 -0.19287723  0.34476324 -0.29997436
 -0.3718064  -0.14502552 -0.3552233  -0.00935247  0.04592827 -0.34122484
  0.15548916 -0.02214367 -0.31110499  3.65527469  0.10811119  3.65527469]


...

30/30 [==============================] - 0s 13ms/step


2500it [00:00, 101951.97it/s]

[6, 7, 5, 7, 7, 1, 5, 9, 5, 8, 8, 8, 8, 7, 7, 4, 7, 0, 0, 3, 8, 6, 1, 1, 8, 9, 11, 3, 11, 8]
[ 2.34968535  1.85013242  1.46791575  1.85013242  2.34968535 -0.52408433
 -0.90186174 -0.79355447 -1.15461074 -1.11957195 -1.15103021 -1.01779899
 -1.00746045 -0.67587317 -0.47453763 -0.45010781 -0.21611622  0.13065476
  0.26795153  0.62277154  0.93577803  0.57301744  0.52301634  0.57301744
  0.93577803  1.24282697  1.75496719  1.24282697  1.75496719  2.34968512]
 6/20 [========>.....................] - ETA: 0s - loss: 0.6223 - value_loss: 0.0816 - policy_loss: 0.5408 - policy_acc: 0.7214

20/20 [==============================] - 0s 12ms/step - loss: 0.5863 - value_loss: 0.0875 - policy_loss: 0.4988 - policy_acc: 0.7220


 70%|███████   | 1762/2500 [00:47<00:20, 36.38it/s]